In [1]:
import torch
from torch.autograd import Variable
from torch.autograd import Function
from torchvision import models
from torchvision import utils
import cv2
import sys
import numpy as np
import argparse
import torch.nn as nn
import pdb 
import torch.utils.data as data
import h5py
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
import PIL.Image as Image
%matplotlib inline 

/home/aashi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class FrameDataset(data.Dataset):
    
    def __init__(self, f, transform=None):
        self.f = f 
        self.transform = transform 
        
    def __getitem__(self, index):
        rgb = np.array(self.f["rgb"][index])
        label = np.array(self.f["labels"][index], dtype=np.uint8)
#         scene_num = self.f["scene_num"][index]
#         frame_num = self.f["frame_num"][index]
        
        if (label > 0):
            label = 1
        
        ## label value transformed to binary vector
        t_label = torch.zeros(2)
        t_label[label] = 1
        
        t_rgb = torch.zeros(rgb.shape[0], 3, 224, 224)
        
        if self.transform is not None: ## Have to create a video for non transform 
            for i in range(rgb.shape[0]):
#                 img_to_resize = rgb[i,:,:,:]
#                 img_to_resize = Image.fromarray(img_to_resize.astype('uint8'), 'RGB')
#                 img_to_resize = img_to_resize.resize((224, 224))    
#                 img_to_resize = self.transform(img_to_resize)
                t_rgb[i,:,:,:] = self.transform(rgb[i,:,:,:])
                
#         return rgb, t_rgb, scene_num, frame_num, t_label ## also returning rgb for visualization 
        return rgb, t_rgb, t_label 
    
    def __len__(self):
        return len(self.f["rgb"])

In [3]:
def load_vgg_voc_weights(MODEL_PATH):
    checkpoint_dict = torch.load(MODEL_PATH)
    vgg_model.load_state_dict(checkpoint_dict)

vgg_model = models.vgg16(pretrained=True)
num_final_in = vgg_model.classifier[-1].in_features
NUM_CLASSES = 20 ## in VOC
vgg_model.classifier[-1] = nn.Linear(num_final_in, NUM_CLASSES)
model_path = '/home/aashi/the_conclusion/model_files/' + 'vgg_on_voc' + str(800)
load_vgg_voc_weights(model_path)

class VGGNet(nn.Module):
    
    def __init__(self):
        super(VGGNet, self).__init__()
        self.rgb_net = self.get_vgg_features()
        
        kernel_size = 3 
        padding = int((kernel_size - 1)/2)
        self.conv_layer = nn.Conv2d(512, 16, kernel_size, 1, padding, bias=True)
        ## input_channels, output_channels, kernel_size, stride, padding, bias
        self.feature_size = 16*7*7*4
        self.final_layer = nn.Sequential(
        nn.Linear(self.feature_size, 256),
        nn.Linear(256, 2),
        #nn.Sigmoid()
        nn.Softmax() 
        )
        
    def forward(self, rgb): ## sequence of four images - last index is latest 
        four_imgs = []
        for i in range(rgb.shape[1]):
            img_features = self.rgb_net(rgb[:,i,:,:,:])
            channels_reduced = self.conv_layer(img_features)
            img_features = channels_reduced.view((-1, 16*7*7))
            four_imgs.append(img_features)
        concat_output = torch.cat(four_imgs, dim = 1)
        out = self.final_layer(concat_output)
        return out
#         return concat_output
        
    def get_vgg_features(self):

        ##vgg16 = load_vgg_voc_weights(vgg16, model_path)
        modules = list(vgg_model.children())[:-1]
        ## I can also freeze 
        ## high level layer, should I take a lower level?
        vgg16 = nn.Sequential(*modules)
        
        ## Uncommented this to let it fine-tune on my model 
        # for p in vgg16.parameters():
        #     p.requires_grad = False 
        
        return vgg16.type(torch.Tensor)

In [4]:
class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """
    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
    	self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        #pdb.set_trace()
        for name, module in self.model._modules.items():
            #pdb.set_trace()
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

In [5]:
class ModelOutputs():
    """ Class for making a forward pass, and getting:
    1. The network output.
    2. Activations from intermeddiate targetted layers.
    3. Gradients from intermeddiate targetted layers. """
    def __init__(self, model, target_layers):
        self.model = model
        self.feature_extractor = FeatureExtractor(self.model.rgb_net, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x): ## x is a sequence of four imgs [1,4,3,224,224] 
        four_imgs = []
        target_activations, _ = self.feature_extractor(x[:,0,:,:,:])
#         pdb.set_trace()
        for i in range(x.shape[1]):
            _, output  = self.feature_extractor(x[:,i,:,:,:])
            #pdb.set_trace()
            channels_reduced = self.model.conv_layer(output)
            img_features = channels_reduced.view((-1, 16*7*7))
            four_imgs.append(img_features)
        #pdb.set_trace()
        concat_output = torch.cat(four_imgs, dim = 1)
#             output = output.view(output.size(0), -1)
        out = self.model.final_layer(concat_output) ## Have to fix the
#         print(out)
        return target_activations, out ## target_activation of latest image

In [6]:
def show_cam_on_image(img, mask, seq):
	heatmap = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
	heatmap = np.float32(heatmap) / 255
	cam = heatmap + np.float32(img)
	cam = cam / np.max(cam)
# 	pdb.set_trace()    
# 	cv2.imwrite("heatmaps/" + str(iter + 1) + ".jpg", np.uint8(255 * cam))
# 	cv2.imwrite("heatmaps_test_seq_2/" + str(iter+1) + ".jpg", np.uint8(255 * cam))
	cv2.imwrite("heatmaps_paper/" + str(iter+1) + ".jpg", np.uint8(255 * cam))

In [7]:
class GuidedBackpropReLU(Function):

    def forward(self, input):
        positive_mask = (input > 0).type_as(input)
        output = torch.addcmul(torch.zeros(input.size()).type_as(input), input, positive_mask)
        self.save_for_backward(input, output)
        return output

    def backward(self, grad_output):
        input, output = self.saved_tensors
        grad_input = None

        positive_mask_1 = (input > 0).type_as(grad_output)
        positive_mask_2 = (grad_output > 0).type_as(grad_output)
        grad_input = torch.addcmul(torch.zeros(input.size()).type_as(input), torch.addcmul(torch.zeros(input.size()).type_as(input), grad_output, positive_mask_1), positive_mask_2)

        return grad_input

In [8]:
class GuidedBackpropReLUModel:
	def __init__(self, model, use_cuda):
		self.model = model
		self.model.eval()
		self.cuda = use_cuda
		if self.cuda:
			self.model = model.cuda()

		# replace ReLU with GuidedBackpropReLU
		for idx, module in self.model.features._modules.items():
			if module.__class__.__name__ == 'ReLU':
				self.model.features._modules[idx] = GuidedBackpropReLU()

	def forward(self, input):
		return self.model(input)

	def __call__(self, input, index = None):
		if self.cuda:
			output = self.forward(input.cuda())
		else:
			output = self.forward(input)

		if index == None:
			index = np.argmax(output.cpu().data.numpy())

		one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
		one_hot[0][index] = 1
		one_hot = Variable(torch.from_numpy(one_hot), requires_grad = True)
		if self.cuda:
			one_hot = torch.sum(one_hot.cuda() * output)
		else:
			one_hot = torch.sum(one_hot * output)

		# self.model.features.zero_grad()
		# self.model.classifier.zero_grad()
		one_hot.backward()

		output = input.grad.cpu().data.numpy()
		output = output[0,:,:,:]

		return output

In [9]:
class GradCam:
	def __init__(self, model, target_layer_names, use_cuda):
		self.model = model
		self.model.eval()
		self.cuda = use_cuda
		if self.cuda:
			self.model = model.cuda()

		self.extractor = ModelOutputs(self.model, target_layer_names)

	def forward(self, input):
		return self.model(input) 

	def __call__(self, input, index = None):
		if self.cuda:
			features, output = self.extractor(input.cuda())
		else:
			features, output = self.extractor(input)

		if index == None:
			index = np.argmax(output.cpu().data.numpy())

		one_hot = np.zeros((1, output.size()[-1]), dtype = np.float32)
		one_hot[0][index] = 1
		one_hot = Variable(torch.from_numpy(one_hot), requires_grad = True)
		if self.cuda:
			one_hot = torch.sum(one_hot.cuda() * output)
		else:
			one_hot = torch.sum(one_hot * output)

		self.model.rgb_net.zero_grad()
		self.model.conv_layer.zero_grad()
		self.model.final_layer.zero_grad()
		one_hot.backward()

		grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

		target = features[-1]
		target = target.cpu().data.numpy()[0, :]

		weights = np.mean(grads_val, axis = (2, 3))[0, :]
		cam = np.zeros(target.shape[1 : ], dtype = np.float32)

		for i, w in enumerate(weights):
			cam += w * target[i, :, :]        

		cam = np.maximum(cam, 0)        ## ReLU here ## only the pixels which have positive influence on the class 
		cam = cv2.resize(cam, (224, 224))
		cam = cam - np.min(cam) ## Normalized for visualization purpose 
		cam = cam / np.max(cam)
		return cam

In [10]:
def preprocess_image(imgs):
    
    t_rgb = torch.zeros(4,3,224,224)

    means=[0.485, 0.456, 0.406]
    stds=[0.229, 0.224, 0.225]

    for i in range(4):
        img = imgs[i]
        preprocessed_img = img.copy()[: , :, ::-1]
        for i in range(3):
            preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
            preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
        preprocessed_img = \
            np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))
        preprocessed_img = torch.from_numpy(preprocessed_img)
        #preprocessed_img.unsqueeze_(0)
        t_rgb[i,:,:,:] = preprocessed_img 
    t_rgb.unsqueeze_(0)
    input = Variable(t_rgb, requires_grad = True)
    return input

In [11]:
def get_args():
	parser = argparse.ArgumentParser()
	parser.add_argument('--use-cuda', action='store_true', default=False,
	                    help='Use NVIDIA GPU acceleration')
	parser.add_argument('--image-path', type=str, default='./examples/both.png',
	                    help='Input image path')
	args = parser.parse_args()
	args.use_cuda = args.use_cuda and torch.cuda.is_available()
	if args.use_cuda:
	    print("Using GPU for acceleration")
	else:
	    print("Using CPU for computation")

	return args

In [12]:
def load_model_weights(MODEL_PATH):
    checkpoint_dict = torch.load(MODEL_PATH)
    model.load_state_dict(checkpoint_dict)

In [13]:
""" python grad_cam.py <path_to_image>
1. Loads an image with opencv.
2. Preprocesses it for VGG19 and converts to a pytorch variable.
3. Makes a forward pass to find the category index with the highest score,
and computes intermediate activations.
Makes the visualization. """

# args = get_args()
use_cuda = 1
image_path_1 = 'examples/611.png'
image_path_2 = 'examples/612.png'
image_path_3 = 'examples/613.png'
image_path_4 = 'examples/614.png'

# Can work with any model, but it assumes that the model has a 
# feature method, and a classifier method,
# as in the VGG models in torchvision.

# object_categories = ['aeroplane', 'bicycle', 'bird', 'boat',
#                      'bottle', 'bus', 'car', 'cat', 'chair',
#                      'cow', 'diningtable', 'dog', 'horse',
#                      'motorbike', 'person', 'pottedplant',
#                      'sheep', 'sofa', 'train', 'tvmonitor']

# model = models.vgg16(pretrained=True)
model = VGGNet()
# num_final_in = model.classifier[-1].in_features
# NUM_CLASSES = len(object_categories)
# model.classifier[-1] = nn.Linear(num_final_in, NUM_CLASSES)

## model = model.cuda()

######## I deleted the lidar_jitter model files ########
# model_path = '/mnt/hdd1/aashi/lidar_jitter_025'  ## Unfortunately I deleted this 

## New Model trained with similar loss 
model_path = '/mnt/hdd1/aashi/lidar_jitter_014'
# model_path = '/home/aashi/the_conclusion/model_files/' + 'vgg_on_voc' + str(800)
load_model_weights(model_path)

# grad_cam = GradCam(model = models.vgg19(pretrained=True), \
# 				target_layer_names = ["35"], use_cuda=args.use_cuda)

print(model)

# model2 =  models.vgg19(pretrained=True)
# print(model2)

grad_cam = GradCam(model = model, target_layer_names = ["0"], use_cuda=use_cuda)

# img = cv2.imread(image_path, 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# input = preprocess_image(img)

# list_of_imgs = []
# img = cv2.imread(image_path_1, 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# #img = np.fliplr(img)
# img1 = img
# list_of_imgs.append(img)

# img = cv2.imread(image_path_2, 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# #img = np.fliplr(img)
# img2 = img
# list_of_imgs.append(img)

# img = cv2.imread(image_path_3, 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# #img = np.fliplr(img)
# img3 = img
# list_of_imgs.append(img)

# img = cv2.imread(image_path_4, 1)
# img = np.float32(cv2.resize(img, (224, 224))) / 255
# #img = np.fliplr(img)
# img4 = img
# list_of_imgs.append(img)
# input = preprocess_image(list_of_imgs)

# hfp_test = h5py.File('/mnt/hdd1/aashi/cmu_data/complete_data_test.h5', 'r')


hfp_test = h5py.File('/mnt/hdd1/aashi/cmu_data/test_seq_2.h5', 'r')

# hfp_test = h5py.File('/mnt/hdd1/aashi/cmu_data/test_seq_2_high_res.h5', 'r')
# grad CAM not good 

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
batch_size = 1
transform = transforms.Compose([transforms.ToTensor(), normalize])
test_loader = data.DataLoader(FrameDataset(f = hfp_test, transform = transforms.Compose(
    [transforms.ToTensor(), normalize])), batch_size = batch_size)

# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
#target_index = None
# Changing the target index from None to 1; 1 corresponds to pred = 1 
# changes the target to None - backpropragate high prob score 
target_index = None 

# mask = grad_cam(input, target_index)
# show_cam_on_image(img1, mask, 1)

for iter, (rgb, t_rgb, label) in enumerate(test_loader, 0):
    input = t_rgb.float().cuda()
    mask = grad_cam(input, target_index)
    vrgb = rgb[0,3,:,:,:]  ## heatmap of last image 
    img = np.ascontiguousarray(vrgb)

#     resized_mask = cv2.resize(mask, (1280, 720))
#     show_cam_on_image(img/255, resized_mask, iter)
    show_cam_on_image(img/255, mask, iter)
    print(iter)
    if (iter == 65):
        pdb.set_trace()
# pdb.set_trace()

# gb_model = GuidedBackpropReLUModel(model = models.vgg19(pretrained=True), use_cuda=use_cuda)
# gb = gb_model(input, index=target_index)
# utils.save_image(torch.from_numpy(gb), 'gb2.jpg')

# cam_mask = np.zeros(gb.shape)
# for i in range(0, gb.shape[0]):
#     cam_mask[i, :, :] = mask

# cam_gb = np.multiply(cam_mask, gb)
# utils.save_image(torch.from_numpy(cam_gb), 'cam_gb2.jpg')

VGGNet(
  (rgb_net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace)
      (16): MaxPool2d(kernel_size=2, stride=2,

/home/aashi/anaconda3/envs/cenv/lib/python3.6/site-packages/torch/nn/modules/container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
> <ipython-input-13-3da5dc6e65ef>(105)<module>()
-> for iter, (rgb, t_rgb, label) in enumerate(test_loader, 0):
(Pdb) q


BdbQuit: 

In [ ]:
########## What is my classification layer? ###########

########## This is BINARY CLASSIFICATION ###########

# hfp_test = h5py.File('/mnt/hdd1/aashi/cmu_data/test_seq_2.h5', 'r')

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# batch_size = 1
# transform = transforms.Compose([transforms.ToTensor(), normalize])
# test_loader = data.DataLoader(FrameDataset(f = hfp_test, transform = transforms.Compose(
#     [transforms.ToTensor(), normalize])), batch_size = batch_size)

# for iter, (rgb, t_rgb, label) in enumerate(test_loader, 0):
#     pdb.set_trace()